I have hourly data for geographical positions, speed over goround, and CO2 emissions from 100 unique container ships for the year 2019. I have also loaded a list of major ports across the world. Assuming the sample is complete and representative, I intened to present a simple analysis of air pollution around ports. 

In [75]:
import pandas as pd
import sqlite3

# Reading the data

In [76]:
con = sqlite3.connect("dwh/db")

ships = pd.read_sql_query('select * from ships', con)
ports = pd.read_sql_query('select * from ports', con)
ports_emissions = pd.read_sql_query('select * from ports_emissions', con)

DatabaseError: Execution failed on sql 'select * from ports_emissions': no such table: ports_emissions

             closest_port continent      E_CO2_kg   LATITUDE   LONGITUDE
0              ABU RUDEIS    Africa  2.211624e+06  28.907334   33.183100
1              ABU ZENIMA    Africa  1.364489e+04  29.043134   33.119100
2                ABURATSU      Asia  3.066918e+04  31.571434  131.406500
3                 ADABIYA    Africa  8.560927e+06  29.865334   32.473500
4                    ADEN      Asia  7.887054e+04  12.795288   45.005310
..                    ...       ...           ...        ...         ...
559                 ZHAPU      Asia  5.244889e+04  30.583333  121.083333
560              ZHEN HAI      Asia  2.777393e+04  29.950000  121.700000
561              ZHOUSHAN      Asia  2.541532e+07  30.006934  122.098700
562                ZHUHAI      Asia  7.450811e+05  22.233333  113.583333
563  ZONA FRANCA DE CADIZ    Europe  1.647433e+04  36.503513   -6.263381

[564 rows x 5 columns]


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Assuming 'ports' is a GeoDataFrame with geometries and emissions data
# Assuming 'E_CO2_kg' is the column with emissions data

# Plot the world map
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
fig, ax = plt.subplots(figsize=(10, 6))
world.plot(ax=ax, color='lightgrey')

# Plot ports with bubble size representing emissions
for index, port in ports.iterrows():
    lat = port['Lat']
    lon = port['Lng']
    
    # You may want to customize the bubble size based on emissions
    bubble_size = port['E_CO2_kg'] * 0.0001
    
    # Plot the bubble
    ax.scatter(lon, lat, s=bubble_size, color='blue', alpha=0.7, edgecolors='black')
    
    # Annotate with port information
    ax.text(lon, lat, f"{port['Postal']} ({port['E_CO2_kg']} kg)", fontsize=8)

plt.title('Emissions at Ports')
plt.show()
